In [1]:
import numpy as np
import matplotlib.pyplot as plt
from emitter import Emitter

In [2]:
cmin = 0
cmax = 2
n_points = 20

In [3]:
c = np.linspace(cmin, cmax, n_points)

In [4]:
grid = np.array([(c[i], c[j]) for j in range(n_points) for i in range(n_points)])

In [5]:
y = lambda x: .2 * x ** 3 - .3 * x + .3
fx = y(c)

In [6]:
class Cube:
    def __init__(self, coords, state="empty"):
        """ Cube is defined by a down-bottom-left point """
        self.c = coords.copy()
        self.state = state

    def __repr__(self):
        return f"[{self.coords.round(2)}] - {self.state}"

    @property
    def coords(self):
        return self.c

    @coords.setter
    def coords(self, val):
        self.c = val

    def is_empty(self):
        return self.state == 'empty'

class HeightMap:
    def __init__(self, cmin, cmax, n_points, n_dims=2, estimator=None):
        self.n = n_points
        self.n_dims = n_dims
        self.hmap = np.empty((self.n ** n_dims,), dtype=object)
        
        self.c = np.linspace(cmin, cmax, n_points)
        self.step = c[1] - c[0]
        
        for i in range(self.n ** n_dims):
            idx = divmod(i, n_points)
            cp = np.array([c[_] for _ in idx])
            self.hmap[i] = Cube(cp, 'terrain' if self.under_f(y, cp) else 'empty')
    
    def dn(self, params):
        """ n-dimentional index convertion to 1D array """
        n = len(params)
        return self.hmap[sum([_ * self.n ** p for _, p in zip(params, range(n - 1, -1, -1))])]
        
    
    def i_to_ndim_idx(i):
        return tuple([i])
    
    def under_f(self, f, point):
        f_val = f(point[:-1])
        return point[-1] <= f_val
    
    def reset(self):
        for cube in self.hmap:
            if cube.state == 'water':
                cube.state = 'empty'

In [7]:
hmap = HeightMap(0, 2, 20)
print(hmap.hmap.shape)
e = Emitter(cmin, cmax, n_coords=2)
# e.set_origin(np.array((.8, 1)))

(400,)


In [8]:
class Simulation:
    def __init__(self, cmin=0, cmax=2, n_points=20):
        self.particles = []
        self.dt = 0.003
        self.cmin = cmin
        self.cmax = cmax
        self.n_grid = n_points
        self.c = np.linspace(cmin, cmax, n_points)
        self.step = self.c[1] - self.c[0]
    
    def add_particles(self, particles):
        self.particles.extend(particles)
    
    def add_particle(self, particle):
        self.particles.append(particle)
    
    def update_particles(self):
        # 1) gravity
        for particle in self.particles:
            cur_loc = self.discritize(particle)
            next_loc = self.discritize(particle + particle.grad)
#             print(f"Cur loc: {cur_loc} {hmap.dn(cur_loc).state} | Next loc: {next_loc} {hmap.dn(next_loc).state} | grad: {particle.grad}")
#             print(f"Next loc state: {hmap.dn(next_loc).state}")
            if hmap.dn(next_loc).state != 'empty':
                ground_level = next_loc
                # ground is somewhere between cur_loc and next_loc
                for i in range(cur_loc[1] - 1, next_loc[1] - 1, -1):
#                     print(f"Loc: {cur_loc[0], i} - {hmap.dn((cur_loc[0], i)).state}")
                    if hmap.dn((cur_loc[0], i)).state != 'empty':
                        next_loc = (cur_loc[0], i + 1)
                        break
#                 print("Corrected next loc:", next_loc)
                nloc = hmap.dn(next_loc)
                particle.coords = nloc.coords
                particle.grad += -particle.grad
                nloc.state = 'water'
            else:
                particle.grad[((particle.coords + particle.grad) < self.cmin) | ((particle.coords + particle.grad) > self.cmax)] = 0
                particle.coords += particle.grad
                particle.grad[-1] += -9.81 * self.dt
    
    def discritize(self, particle):
        return [int(_ / self.step) for _ in particle.coords]
    
    def reset(self):
        self.particles = []

In [9]:
s = Simulation()
# s.add_particles([_.copy() for _ in particles])
s.particles

[]

In [10]:
import ipywidgets as widgets

coords = np.array([hmap.hmap[i].coords for i in range(hmap.n ** 2)])
col = ['green' if hmap.hmap[i].state == 'terrain' else 'blue' for i in range(hmap.n ** 2)]

def f(t):
    if t == 0:
        s.reset()
#         s.add_particles([_.copy() for _ in particles])
        e.reset()
        hmap.reset()
    s.update_particles()
    p = e.emit()
    if p:
        s.add_particle(p)
    plt.scatter(coords[:, 0], coords[:, 1], s=1, c=col, label="terrain");
    plt.scatter([p.x for p in s.particles], [p.y for p in s.particles], c='blue', label="particle")
    plt.plot(c, fx, c='orange', label="Surface function");
    plt.xlim(cmin, cmax);
    plt.ylim(cmin, cmax);
    plt.legend(loc="upper left");
# @widgets.interact(t=widgets.FloatSlider(min=0.0, max=2.0, value=0), continuous_update=False)

ip = widgets.interactive(f, t=widgets.FloatSlider(min=0.0, max=4.0, value=0), continuous_update=False)
ip.layout.height = '350px'
ip

interactive(children=(FloatSlider(value=0.0, description='t', max=4.0), Output()), layout=Layout(height='350px…